# Chemical equilibrium with constraints

In this tutorial, we demonstrate that Reaktoro can be used for wide range of equilibrium problems and explain how to

* *specify different equilibrium constraints* when calculating chemical equilibrium; and
* *configure equilibrium solver* to be open to one or more substances.

Let us re-call the simplest chemical equilibrium discussed earlier:

In [2]:
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

solver = EquilibriumSolver(system)

We re-call that [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) objects created the way above are limited to chemical equilibrium calculations where:
* T and P are given and
* the system is closed (no mass transfer in or out of the system).

**But!** To impose the temperature and volume of the system (the class of *Helmholtz energy minimization problems*), we need to construct a specialized [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) object that handles alternative constraint specifications.

> **Note**: Instead of the Helmholtz energy minimization problem, Reaktoro implements a single **parametric Gibbs energy minimization solver that accepts general equilibrium constraints**, which can be configured to efficiently solve all other classes of equilibrium problems.


## Specifying constrained properties

Let's create an equilibrium solver that can solve problems with prescribed **temperature** and **volume**:

In [3]:
# Define equilibrium specification class indicating fixed conditions in the minimization problem
specs = EquilibriumSpecs(system)
specs.temperature()
specs.volume()

# Defining equilibrium solver using above-defined equilibrium specifications
solver = EquilibriumSolver(specs)

We can use [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) to provide the constraint specifications for our [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html).

> **Tip**: Beside the list of possible constraints (one can impose) with [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) class, it is **possible to define custom constraints**.

With the chemical system and chemical solver defined above, we want to model the combustion of 1 mol of CH<sub>4</sub> and 0.5 mol of O<sub>2</sub> in a chamber of 10 cm³ at 1000 °C:

In [4]:
state = ChemicalState(system)
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+--------+------+
| Property        |  Value | Unit |
+-----------------+--------+------+
| Temperature     | 298.15 |    K |
| Pressure        | 100000 |   Pa |
| Charge:         |      0 |  mol |
| Element Amount: |        |      |
| :: H            |      4 |  mol |
| :: C            |      1 |  mol |
| :: O            |      1 |  mol |
| Species Amount: |        |      |
| :: CH4          |      1 |  mol |
| :: O2           |    0.5 |  mol |
| :: CO2          |  1e-16 |  mol |
| :: CO           |  1e-16 |  mol |
| :: H2O          |  1e-16 |  mol |
| :: H2           |  1e-16 |  mol |
+-----------------+--------+------+


This initial state is still in disequilibrium. We use [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html) below to specify the desired conditions of temperature and volume at the equilibrium state and equilibrate the `state` object:

In [5]:
# Specify that the values for fixed temperature and volume constraints
conditions = EquilibriumConditions(specs)
conditions.temperature(1000.0, "celsius")
conditions.volume(10.0, "cm3")

# Equilibrate the chemical state
solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |     1273.15 |    K |
| Pressure        | 1.66032e+09 |   Pa |
| Charge:         |           0 |  mol |
| Element Amount: |             |      |
| :: H            |           4 |  mol |
| :: C            |           1 |  mol |
| :: O            |           1 |  mol |
| Species Amount: |             |      |
| :: CH4          |     0.71576 |  mol |
| :: O2           |       1e-16 |  mol |
| :: CO2          |    0.225093 |  mol |
| :: CO           |   0.0591477 |  mol |
| :: H2O          |    0.490667 |  mol |
| :: H2           |   0.0778142 |  mol |
+-----------------+-------------+------+


To make sure that the final volume in the chemical state is indeed 10 cm³, we can use the [ChemicalProps](https://reaktoro.org/api/classReaktoro_1_1ChemicalProps.html) object attached to the [ChemicalState](https://reaktoro.org/api/classReaktoro_1_1ChemicalState.html) object `state` by [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) at the end of the minimization:

In [6]:
V = state.props().volume()  # volume in m³
P = state.pressure()        # pressure in Pa

V = units.convert(V, "m3", "cm3")  # convert volume from m³ to cm³
P = units.convert(P, "Pa", "bar")  # convert pressure from Pa to bar

print("Volume at equilibrium state is", V, "cm³")
print("Pressure at equilibrium state is", P, "bar")

Volume at equilibrium state is 10 cm³!
Pressure at equilibrium state is 1.66032 GPa!


**Important:** for every condition marked to be known in the [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) object, we must set the expected conditions in the [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html) object! Not doing so can cause unexpected behavior or a runtime error.

**TASK**: reformulate the Gibbs energy minimization problem for modelling the combustion of 1 mol of CH<sub>4</sub> and 0.5 mol of O<sub>2</sub> at 1000 ºC and 100 bar with [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) and [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html).

In [1]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #
from reaktoro import *

db = NasaDatabase("nasa-cea")

gases = GaseousPhase("CH4 O2 CO2 CO H2O H2")

system = ChemicalSystem(db, gases)

specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()

solver = EquilibriumSolver(specs)

conditions = EquilibriumSpecs(specs)
conditions.temperature(1000, "celsius")
conditions.pressure(100, "bar")


state = ChemicalState(system)
state.set("CH4", 1.0, "mol")
state.set("O2",  0.5, "mol")

solver.solve(state, conditions)

print(state)

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. reaktoro.reaktoro4py.EquilibriumSpecs(arg0: reaktoro.reaktoro4py.ChemicalSystem)

Invoked with: <reaktoro.reaktoro4py.EquilibriumSpecs object at 0x7fb9ab2b13b0>

## Specifying open conditions

Assume, we need to find out how many mols of CH<sub>4</sub> must react with 0.5 mol of O<sub>2</sub> in the same chamber (with volume 10 cm³ and temperature 1000 °C) to obtain pressure 1 GPa (maximum pressure supported by the chamber material). In such equilibrium calculation, the system must be assumed to be open to the mass transfer of CH<sub>4</sub>, which creates a new *degree of freedom*: the amount of added/removed substance.

We start with the chemical state containing only 0.5 mols of O<sub>2</sub>:

In [7]:
state = ChemicalState(system)
state.set("O2", 0.5, "mol")

Next, we create an [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) object specifying **temperature**, **volume**, and **pressure** as constrained properties. We also specify that the system is open to CH<sub>4</sub>.

In [8]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.volume()
specs.pressure()
specs.openTo("CH4")

Then, we create the [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html) object, providing the values for temperature, volume, and pressure, and perform the equilibrium calculation:

In [9]:
conditions = EquilibriumConditions(specs)
conditions.temperature(1000.0, "celsius")
conditions.volume(10.0, "cm3")
conditions.pressure(1.0, "GPa")

solver = EquilibriumSolver(specs)
solver.solve(state, conditions)

print(state.props())

+----------------------------------------+-------------+-----------+
| Property                               |       Value |      Unit |
+----------------------------------------+-------------+-----------+
| Temperature                            |     1273.15 |         K |
| Pressure                               |       1e+09 |        Pa |
| Volume                                 |       1e-05 |        m3 |
| Gibbs Energy                           |     -488420 |         J |
| Enthalpy                               |     -184793 |         J |
| Entropy                                |     238.485 |       J/K |
| Internal Energy                        |     -194793 |         J |
| Helmholtz Energy                       |     -498420 |         J |
| Charge                                 |           0 |       mol |
| Element Amount:                        |             |           |
| :: H                                   |     1.59954 |       mol |
| :: C                            

The above table shows that the obtained equilibrium state meats desired values for temperature, pressure, and volume (in SI units). We can output the amount of CH<sub>4</sub> entering the system so that all prescribed conditions could be attained:

In [10]:
print("Amount of CH4 titrated in:", state.equilibrium().explicitTitrantAmounts()[0], "mol")

Amount of CH4 titrated in: 0.3998837695340861 mol
